In [1]:
#include <iostream>
#include <span>
#include <vector>
#include <array>
#include <numeric>
#include <type_traits>
#include <cmath>
#include <memory>
#include <stdio.h> 

In [2]:
template<typename T>
constexpr std::string_view type_name() {
    std::string_view name, prefix, suffix;
#ifdef __clang__
    name = __PRETTY_FUNCTION__;
    prefix = "std::string_view type_name() [T = ";
    suffix = "]";
#elif defined(__GNUC__)
    name = __PRETTY_FUNCTION__;
    prefix = "constexpr std::string_view type_name() [with T = ";
    suffix = "; std::string_view = std::basic_string_view<char>]";
#elif defined(_MSC_VER)
    name = __FUNCSIG__;
    prefix = "class std::basic_string_view<char,struct std::char_traits<char> > __cdecl type_name<";
    suffix = ">(void)";
#endif
    name.remove_prefix(prefix.size());
    name.remove_suffix(suffix.size());
    return name;
}

## 2.1.5 SFINAE

In [3]:
template<typename T> T declval_(long);
template<typename T, typename U=T&&> U declval_(int);

In [4]:
template<typename T> auto declval() -> decltype(declval_<T>(0)){
    static_assert("Cannot be used in evaluation expression");
    return declval_<T>(0);
}

## 2.2.1 Fibonacci

In [5]:
template<size_t N> constexpr size_t Fibonacci = Fibonacci<N-1> + Fibonacci<N-2>;
template<> constexpr size_t Fibonacci<0> = 0;
template<> constexpr size_t Fibonacci<1> = 1;

In [6]:
std::cout << "Fibonacci<10> is " << Fibonacci<10> << std::endl;
std::cout << "Fibonacci<2> is " << Fibonacci<2> << std::endl;

Fibonacci<10> is 55
Fibonacci<2> is 1


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe63c817500


In [7]:
template<size_t N> constexpr double golden_ratio = Fibonacci<N+1> * 1.0 / Fibonacci<N>;

In [8]:
std::cout.precision(std::numeric_limits<double>::max_digits10);
std::cout << golden_ratio<20> << std::endl;
std::cout << golden_ratio<50> << std::endl;

1.6180339985218033
1.6180339887498949


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe63c817500


## 2.2.2 Type Traits

In [9]:
void PrintArrayLike(std::span<int> container){
    std::cout << "size: " << container.size() << std::endl;
    for(auto elem : container){
        std::cout << elem << std::endl;
    }
}

In [10]:
std::array arr{1,2,3,4};
std::vector v{1,2,3};

In [11]:
PrintArrayLike(arr);

size: 4
1
2
3
4


(void) nullptr


In [12]:
PrintArrayLike(v);

size: 3
1
2
3


(void) nullptr


In [13]:
using cint = const int;

In [14]:
std::cout << type_name<cint>() << std::endl;

const int


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe63c817500


In [15]:
using dvector = std::vector<double>;
std::cout << type_name<dvector>() << std::endl;

std::vector<double>


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe63c817500


## 2.2.3 辅助类

In [16]:
using TWO = std::integral_constant<int, 2>;
using FOUR = std::integral_constant<int, 4>;

In [17]:
std::cout << type_name<TWO::value_type>() << std::endl;

int


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe63c817500


In [18]:
std::cout << std::integral_constant<int, 2>::value << std::endl;
std::cout << TWO::value << std::endl;

2
2


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe63c817500


In [19]:
static_assert(TWO::value * TWO::value == FOUR::value);
std::cout << "TWO::value is " << TWO::value << std::endl;

TWO::value is 2


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe63c817500


In [20]:
static_assert(not std::is_same<TWO, FOUR>::value,
                  "TWO and FOUR are equal!");

## 2.2.4 Empty Base Class Optimization

In [21]:
struct Base0{};


In [22]:
std::cout << "size of Base0: " << sizeof(Base0) << std::endl;

size of Base0: 1


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe63c817500


In [23]:
struct Children0{
    Base0 base;
    int dummy;
};

In [24]:
std::cout << "size of Children0: " << sizeof(Children0) << std::endl;

size of Children0: 8


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe63c817500


In [25]:
struct Children1 : Base0{
    int dummy;
};

In [26]:
std::cout << "size of Children1: " << sizeof(Children1) << std::endl;

size of Children1: 4


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe63c817500


In [27]:
struct Children2{
    [[no_unique_address]] Base0 base;
    int dummy;
};

In [28]:
std::cout << "size of Children2: " << sizeof(Children2) << std::endl;

size of Children2: 4


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe63c817500


## 2.2.5 Implementing Type Traits

In [29]:
template<typename T> 
struct remove_const {using type = T; };
template<typename T>
struct remove_const<const T> {using type = T;};

template<typename T>
using remove_const_t = remove_const<T>::type;

In [30]:
std::cout << type_name<remove_const_t<double>>() << std::endl;
std::cout << type_name<remove_const_t<const double>>() << std::endl;

double
double


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe63c817500


## 2.2.6 Type Reflection

In [31]:
template<typename F> struct function_trait;
template<typename Ret, typename...Args> 
struct function_trait<Ret(Args...)> {
    using result_type = Ret;
    using args_type = std::tuple<Args...>;
    static constexpr size_t num_of_args = sizeof...(Args);
    template<size_t I> using arg = std::tuple_element_t<I, args_type>;
};

In [32]:
using test_result_type = function_trait<decltype(PrintArrayLike)>::result_type;
using test_arg_type = function_trait<decltype(PrintArrayLike)>::arg<0>;
constexpr size_t test_n_args = function_trait<decltype(PrintArrayLike)>::num_of_args;

In [33]:
std::cout << type_name<test_result_type>() << std::endl;
std::cout << type_name<test_arg_type>() << std::endl;
std::cout << "number of args: " << test_n_args << std::endl;

void
std::span<int>
number of args: 1


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe63c817500


## 2.2.7 enable_if 

In [34]:
template<bool, typename=void>
struct enable_if{};

template<typename T>  
struct enable_if<true, T> {
    using type = T;
};

template<bool B, typename T=void> 
using enable_if_t = typename enable_if<B, T>::type;

In [35]:
template<typename T, enable_if_t<std::is_integral_v<T>>* = nullptr>
bool numEqual(const T& lhs, const T& rhs){
    std::cout << "comparing using == " << std::endl;
    return lhs == rhs;
}

template<typename T, enable_if_t<std::is_floating_point_v<T>>* = nullptr>
bool numEqual(const T& lhs, const T& rhs){
    std::cout << "comparing floating points" << std::endl;
    return std::fabs(lhs - rhs) < std::numeric_limits<T>::epsilon();
};

In [36]:
numEqual(1,1);

comparing using == 


(bool) true


In [37]:
numEqual(1.0, 1.0);

comparing floating points


(bool) true


## 2.2.8 tag dispatching

In [38]:
template<typename T> 
bool numEqualImpl(const T& lhs, const T& rhs, std::true_type){
    std::cout << "numEqualImpl from true_type tag" << std::endl;
    return std::fabs(lhs - rhs) < std::numeric_limits<T>::epsilon();
}

template<typename T>
bool numEqualImpl(const T& lhs, const T& rhs, std::false_type){
    std::cout << "numEqualImpl from false_type tag" << std::endl;
    return lhs == rhs;
}

In [39]:
template<typename T> 
bool numEqualDispatch(const T& lhs, const T& rhs){
    return numEqualImpl(lhs, rhs, std::is_floating_point<T>{});
}

In [40]:
numEqualDispatch(1.0, 2.0);

numEqualImpl from true_type tag


(bool) false


In [41]:
numEqualDispatch(1, 2);

numEqualImpl from false_type tag


(bool) false


with auto to deduce return type:

In [42]:
template<typename T>
auto numEqualDispatch2(const T& lhs, const T& rhs) -> enable_if_t<std::is_arithmetic_v<T>, bool> {
    return numEqualImpl(lhs, rhs, std::is_floating_point<T>{});
}

In [43]:
numEqualDispatch2(1.0, 2.0);

numEqualImpl from true_type tag


(bool) false


In [44]:
numEqualDispatch2(1, 2);

numEqualImpl from false_type tag


(bool) false


## 2.2.9 if constexpr

In [45]:
template<typename T> 
auto numEqual3(const T& lhs, const T& rhs){
    if constexpr(std::is_integral_v<T>){
        return lhs == rhs;
    }else{
        return std::fabs(lhs - rhs) < std::numeric_limits<T>::epsilon();
    }
}

In [46]:
numEqual3(1.0, 2.0);

(bool) false


In [47]:
numEqual3(1,1);

(bool) true


## 2.2.10 void_t 

In [48]:
template<typename...> using void_t = void;

In [49]:
// main template
template<typename T, typename = void>
struct HasTypeMember : std::false_type{};

In [50]:
// partial specialized
template<typename T>
struct HasTypeMember<T, void_t<typename T::type>> : std::true_type{};

In [51]:
static_assert(not HasTypeMember<int>::value);
static_assert(HasTypeMember<std::true_type>::value);

## 2.3.1 CRTP: visitor code reuse 

In [52]:
struct PrintInfoByPrintf;
struct PrintInfoByCout;

struct Visitor{
    virtual void visit(PrintInfoByCout&) = 0;
    virtual void visit(PrintInfoByPrintf&) = 0;
    virtual ~Visitor() = default;
};

In [53]:
struct PrintInfo{
    virtual void AcceptInfo(Visitor& visitor) = 0;
    virtual ~PrintInfo() = default;
};

In [54]:
template<typename Derived>
struct AutoPrintInfo : PrintInfo{
    void AcceptInfo(Visitor& visitor) override{
        visitor.visit(static_cast<Derived&>(*this));
    }
};

struct PrintInfoByCout : AutoPrintInfo<PrintInfoByCout>{
    void PrintByCout(void){
        std::cout << "PrintByCout called" << std::endl;
    }
};

struct PrintInfoByPrintf : AutoPrintInfo<PrintInfoByPrintf>{
    void PrintByPrintf(void){
        printf("PrintByPrintf called\n");
    }
};



struct PrintVisitor : Visitor{
    void visit(PrintInfoByCout& print){
        print.PrintByCout();
    }
    void visit(PrintInfoByPrintf& print){
        print.PrintByPrintf();
    }
};


In [55]:
PrintVisitor print_visitor;
PrintInfoByCout print_cout;
PrintInfoByPrintf print_printf;

In [56]:
print_cout.AcceptInfo(print_visitor);

PrintByCout called


(void) @0x7fe628ff8030


In [57]:
print_printf.AcceptInfo(print_visitor);

PrintByPrintf called


(void) @0x7fe628ff8030


In [58]:
struct PrintVisitorWithPrefix : Visitor{
    void visit(PrintInfoByCout& print){
        std::cout << "Added some prefix from PrintVisitorWithPrefix" << std::endl;
        print.PrintByCout();
    }
    void visit(PrintInfoByPrintf& print){
        std::cout << "Added some prefix from PrintVisitorWithPrefix" << std::endl;
        print.PrintByPrintf();
    }
};

In [59]:
PrintVisitorWithPrefix print_prefix_visitor;

In [60]:
print_cout.AcceptInfo(print_prefix_visitor);

Added some prefix from PrintVisitorWithPrefix
PrintByCout called


(void) @0x7fe628ff8030


In [61]:
print_printf.AcceptInfo(print_prefix_visitor);

Added some prefix from PrintVisitorWithPrefix
PrintByPrintf called


(void) @0x7fe628ff8030


## 2.3.2 compilation time polymorphism

In [62]:
template <typename Derived> struct Animal {
    void bark() {static_cast<Derived&>(*this).barkImpl(); }
};

In [63]:
class Cat : public Animal<Cat> {
    friend Animal;
    void barkImpl(){
        std::cout << "Miao" << std::endl;
    }
};

class Dog : public Animal<Dog> {
    friend Animal;
    void barkImpl(){
        std::cout << "Wang" << std::endl;
    }
}

In [64]:
Dog dog;
Cat cat;

dog.bark();
cat.bark();

Wang
Miao


(void) @0x7fe628ff8030


## 2.3.3 enable_shared_from_this

In [65]:
struct Obj : std::enable_shared_from_this<Obj> {
    int dummy;
};

In [68]:
auto sp1 = std::make_shared<Obj>();
std::cout << sp1.get() << std::endl;
std::cout << sp1.use_count() << std::endl;
auto sp2 = sp1->shared_from_this();
std::cout << sp2.get() << std::endl;
std::cout << sp2.use_count() << std::endl;

0x7fe60b6621a0
1
0x7fe60b6621a0
2


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe63c817500


## 2.4.1 scalar delayed computation

In [70]:
template<typename T, typename U, typename OP>
struct BinaryExpression{
    BinaryExpression(const T& lhs, const U&, OP op): lhs(lhs), rhs(rhs), op(op){
    
    };
    auto operator()() const {return op(rhs, lhs);};
    protected:
    T lhs;
    U rhs;
    OP op;
};